In [25]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [26]:
filename = "../../data/06/ABox.ttl"

In [27]:
text1 = '''CQ_6.1
What are the titles of the work? What are their types?
'''

query1 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?title ?type ?content
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R19_created_a_realisation_of ?work .
    ?work crm:P102_has_title ?title .
    ?title crm:P2_has_type ?type ;
        crm:P190_has_symbolic_content ?content .
}
'''

In [28]:
text2 = '''CQ_6.2
Which work was created and in which period?
'''

query2 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?work ?start_date ?end_date
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R19_created_a_realisation_of ?work ;
      crm:P4_has_time-span ?time_span .
    ?time_span crm:P82a_begin_of_the_begin ?start_date ;
      crm:P82b_end_of_the_end ?end_date .
}
'''

In [29]:
text3 = '''CQ_6.3
Which works are part of other works? What are the types of the larger works?
'''

query3 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?work ?larger_work ?type
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R19_created_a_realisation_of ?work .
    ?larger_work a lrmoo:F1_Work ;
      lrmoo:R10_has_member ?work ;
      lrmoo:R3_is_realised_in ?larger_exp .
    ?larger_exp lrmoo:R4i_is_embodied_in ?larger_man .
    ?larger_man crm:P2_has_type ?type .
}
'''

In [30]:
text4 = '''CQ_6.4
What is the cultural object about?
'''

query4 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?expression ?subject
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression a lrmoo:F2_Expression ;
      crm:P129_is_about ?subject .
}
'''

In [31]:
text5 = '''CQ_6.5
What is the type of the cultural object?
'''

query5 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?manifestation ?type
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression lrmoo:R4i_is_embodied_in ?manifestation .
    ?manifestation crm:P2_has_type ?type .
}
'''

In [32]:
text6 = '''CQ_6.6
Which license statement is assigned to the cultural object?
'''

query6 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?manifestation ?license ?external_resource
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression lrmoo:R4i_is_embodied_in ?manifestation .
    ?license crm:P2_has_type aat:300435434 ;
      crm:P67_refers_to ?manifestation ;
      crm:P70i_is_documented_in ?external_resource .
}
'''

In [33]:
text7 = '''CQ_6.7
What are the identifiers identifying the cultural object? What are their types?
'''

query7 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?identifier ?type ?content
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression lrmoo:R4i_is_embodied_in ?manifestation .
    ?manifestation lrmoo:R7i_is_exemplified_by ?item .
    ?item crm:P1_is_identified_by ?identifier .
    ?identifier crm:P2_has_type ?type ;
      crm:P190_has_symbolic_content ?content .
}
'''

In [34]:
text8 = '''CQ_6.8
What is the curation activity in which the object is involved? Who carried it out?
'''

query8 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?item ?activity ?actor
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression lrmoo:R4i_is_embodied_in ?manifestation .
    ?manifestation lrmoo:R7i_is_exemplified_by ?item .
    ?activity crm:P16_used_specific_object ?item ;
      crm:P12_occurred_in_the_presence_of ?actor .
}
'''

In [35]:
text9 = '''CQ_6.9
What is the description of the cultural object?
'''

query9 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>

SELECT ?item ?description
WHERE {
    ?creation a lrmoo:F28_Expression_Creation ;
      lrmoo:R17_created ?expression .
    ?expression lrmoo:R4i_is_embodied_in ?manifestation .
    ?manifestation lrmoo:R7i_is_exemplified_by ?item .
    ?item crm:P3_has_note ?description .
}
'''

In [36]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4),
           (text5, query5),
           (text6, query6),
           (text7, query7),
           (text8, query8),
           (text9, query9)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_6.1
What are the titles of the work? What are their types?

+-------------------------------------------------------------------------+-------------------------------------------+--------------------------+
| title                                                                   | type                                      | content                  |
|-------------------------------------------------------------------------+-------------------------------------------+--------------------------|
| https://w3id.org/dharc/ontology/chad-ap/object/data/06/32-work-title-01 | http://vocab.getty.edu/page/aat/300417204 | Essere umano ermafrodita |
| https://w3id.org/dharc/ontology/chad-ap/object/data/06/32-work-title-02 | http://vocab.getty.edu/page/aat/300417207 | Essere umano ermafrodita |
| https://w3id.org/dharc/ontology/chad-ap/object/data/06/32-work-title-02 | http://vocab.getty.edu/page/aat/300417207 | Human hermaphrodite      |
+------------------------------------------------------